[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# Covid-19 Dashboard

Welcome to the Covid-19 Dashboard. 
This dashboard presents a comparison between Covid-19 cases and test in th UK. The data record all Covid tests and cases from the start of the pandemic. All data are access from the official GOV.UK Coronavirus website.

In [5]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [6]:
#help('uk_covid19')
#dir('uk_covid19')

In [7]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [8]:
fil = ['areaType=nation', 'areaName=England']

stru= {"date": "date",
       "cases": "newCasesBySpecimenDate",
       "test":"newVirusTestsBySpecimenDate"}

testnumber= Cov19API(filters=fil, structure=stru).get_json()

with open ("TESTNUMBER.json","wt") as OUTF:
    json.dump(testnumber,OUTF)

In [9]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
    
def wrangle_data(rawdata):
    with open(rawdata, "rt") as INFILE:
        Data=json.load(INFILE)
    Datalist=Data['data']
    
    Dates=[dictionary['date'] for dictionary in Datalist ]
    Dates.sort()
    startdate=parse_date(Dates[0])
    enddate=parse_date(Dates[-1])
    Index=pd.date_range(startdate, enddate, freq='D')
    TESTNUMBERdf=pd.DataFrame(index=Index, columns=['test', 'cases'])

    for i in Datalist:
        date=parse_date(i['date'])
        for column in ['test','cases']:
            if pd.isna(TESTNUMBERdf.loc[date,column]):
                value=float(i[column]) if i[column] != None else 0.0
                TESTNUMBERdf.loc[date,column]=value
    TESTNUMBERdf.fillna(0.0, inplace = True)
    return TESTNUMBERdf

df=wrangle_data('TESTNUMBER.json')


## Download current data

You can download the up to date Covid-19 data with the bottom below. Try it out.
Confirmation message will show up once the data is successfully updated.

In [10]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    new_data=Cov19API(filters=fil, structure=stru).get_json()
    with open ("TESTNUMBER.json","wt") as OUTF:
        json.dump(new_data,OUTF)
    return "TESTNUMBER.json"


In [11]:
status_label = wdg.Label()

def api_button_callback(button):
    try:
        status_label.value = "Fetching data..."
        apidata = access_api()
        global df
        df = wrangle_data(apidata)
        refresh_graph(series)
        button.icon = "check"
        status_label.value = "Data fetched and graph updated."
    except Exception as e:
        status_label.value = "Error: " + str(e)
        button.icon = "times"   


apibutton=wdg.Button(
    description='Update', 
    disabled=False,
    button_style='info', 
    tooltip="Click to download the updated data",
    icon='download')

apibutton.on_click(api_button_callback)
display(apibutton,status_label)
# run all cells before clicking on this button

Button(button_style='info', description='Update', icon='download', style=ButtonStyle(), tooltip='Click to down…

Label(value='')

## Graphs and Analysis

Below is the graphical presentation of the captured data. Linear and Log graphs are both available. The graph automatically update once the new data is fetched.

In [18]:

def testcase_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    plt.show()
    

def refresh_graph(widget):
    current = widget.value
    widget.value = tuple()
    widget.value = current

series=wdg.SelectMultiple(
    options=['test', 'cases'],
    value=['test','cases'], 
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='linear', 
    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False)

controls=wdg.HBox([scale, series])  
# connect the plotting function and the widget    
graph=wdg.interactive_output(testcase_graph, {'gcols': series, 'gscale': scale})

layout=wdg.VBox([controls,graph])
# actually displays the graph
display(layout)

**(C)Han Shuo Ye** ([ec23893@qmul.ac.uk](mailto:ec23893@qmul.ac.uk) ). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).
Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).